In [1]:
NLP Practical 3

SyntaxError: invalid syntax (<ipython-input-1-ef94ede426c5>, line 1)

In [2]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
import tensorflow as tf

In [5]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
input_text = doc.xpath('//content/text()')
input_label = doc.xpath('//head/keywords/text()')

del doc

In [6]:
talk_sentences = []
talknum = len(input_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', input_text[i])
    temp = re.sub(r'\n', '', input_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [7]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [8]:
talk_sentences_token = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentences_token.append(temp)

In [9]:
talk_sentences_token[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [10]:
input_label_prepro = []

for i in range(len(input_label)):
    temp = re.sub(r' ', '', input_label[i])
    input_label_prepro.append(temp.split(','))
    
#print(input_label_prepro)

In [11]:
labels = []

for i in range(len(input_label_prepro)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label_prepro[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label_prepro[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label_prepro[i]):
        temp[2] = 1
    labels.append(temp)
        
print(input_label_prepro[9])
print(labels[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [12]:
# labels_onehot = np.zeros(len(input_label_prepro), 8)
# for i in range(len(input_label_prepro)):
#     labels_onehot[i][labels[i][0]*4+labels[i][1]*2+labels[i][2]*1] = 1

In [13]:
labels_onehot = []
for i in range(len(input_label_prepro)):
    temp = np.zeros(8)
    temp[labels[i][0]*4+labels[i][1]*2+labels[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [14]:
labels_onehot[:10]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])]

In [15]:
train_data = talk_sentences_token[:1585]
validation_data = talk_sentences_token[1585:1835]
test_data = talk_sentences_token[1835:]

train_labels = labels[:1585]
validation_labels = labels[1585:1835]
test_labels = labels[1835:]

train_labels_onehot = labels_onehot[:1585]
validation_labels_onehot = labels_onehot[1585:1835]
test_labels_onehot = labels_onehot[1835:]

In [16]:
print(np.shape(talk_sentences_token))
print(np.shape(labels))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(validation_data))
print(np.shape(validation_labels))
print(np.shape(validation_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 3)
(2085, 8)
(1585,)
(1585, 3)
(1585, 8)
(250,)
(250, 3)
(250, 8)
(250,)
(250, 3)
(250, 8)


In [17]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(validation_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.61135647  0.07066246  0.06246057  0.00630915  0.17350158  0.05362776
  0.01198738  0.01009464]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]
[ 0.344  0.1    0.132  0.032  0.2    0.128  0.036  0.028]


In [18]:
train_sent = []

for i in range(1585):
    for sent in train_data[i]:
        train_sent.append(sent)
    
print(np.size(train_sent))

183771


In [19]:
from gensim.models import Word2Vec

In [20]:
model_ted = Word2Vec(train_sent, min_count=1, size=100)

In [21]:
'hello' in model_ted.vocab.keys()

True

In [22]:
train_bow = []

for talk in train_data:
    N = 0
    res = np.zeros(100)
    for sent in talk:
        for word in sent:
            if word in model_ted.vocab.keys():
                res += model_ted[word]
                N += 1
    res = res / N
    train_bow.append(res)

In [23]:
train_bow[0]

array([ -4.40911715e-01,  -1.56305378e-01,   4.86207991e-02,
        -1.31064025e-01,   5.26537763e-02,  -2.32653266e-01,
        -3.71403901e-01,   1.98181616e-01,  -1.09710596e-01,
        -3.91428094e-02,   1.62313229e-01,   6.13901740e-01,
        -8.09069324e-01,  -4.92731090e-01,  -3.10690476e-02,
        -4.31658135e-02,   7.90516395e-02,  -5.26039568e-01,
         9.66794779e-02,   4.77136136e-01,  -2.40210482e-01,
        -4.66903560e-01,   2.48389070e-01,   8.75109161e-01,
        -6.36745903e-01,   1.84180628e-01,   6.16071252e-02,
        -3.25912640e-01,  -1.70389712e-01,  -8.59234567e-01,
         2.05942637e-01,   1.62702980e-01,  -1.18115005e-01,
        -1.36019821e-01,  -2.66804856e-01,  -3.41546796e-01,
         3.25170085e-01,   1.78417696e-02,   7.90363193e-01,
        -3.88931256e-01,   6.01414446e-01,   2.59063676e-01,
        -3.10316073e-01,   2.04395620e-01,   3.26113053e-01,
         1.59183598e-01,   1.82482911e-01,  -4.61775000e-01,
         3.55113036e-01,

In [24]:
validation_bow = []

for talk in validation_data:
    N = 0
    res = np.zeros(100)
    for sent in talk:
        for word in sent:
            if word in model_ted.vocab.keys():
                res += model_ted[word]
                N += 1
    res = res / N
    validation_bow.append(res)

In [24]:
test_bow = []

for talk in test_data:
    N = 0
    res = np.zeros(100)
    for sent in talk:
        for word in sent:
            if word in model_ted.vocab.keys():
                res += model_ted[word]
                N += 1
    res = res / N
    test_bow.append(res)